In [1]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI #before: langchain.chat_models
import pandas as pd
from langchain.chains import LLMChain
from langfuse.callback import CallbackHandler
# from langchain.llms import OpenAI
import os
from langchain.chains import SequentialChain
# from openai import organization

dotenv_path = load_dotenv(override=True)

google_api_key = os.environ.get("GOOGLE_API_KEY")
if not google_api_key:
    print("ERROR: GOOGLE_API_KEY not found in environment variables. Please set it in your .env file.")

handler = CallbackHandler(os.environ.get("LANGFUSE_PUBLIC_KEY"), os.environ.get("LANGFUSE_SECRET_KEY"))

C:\Users\Marija\IdeaProjects\effective-gui-prototyping\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL = "gemini-2.0-flash" # before: gpt-4-1106-preview
# ORGANISATION = "org-uZrowMmGKXM2eaDHBuh0Qfs4"

OUTPUT_FOLDER = "generated_guis_mm"
DATASET = "dataset.csv"

df = pd.read_csv(DATASET, sep=";")

In [3]:
from prompts import SUMMARY2REQUIREMENTS_ZS, REQUIREMENTS2IMPLEMENTATION_ZS, IMPLEMETATION2STRUCTURE_ZS, STRUCTURE2CODE_ZS

#PD_ZS
def pd_zs(summary, temp=0.7):
    llm = ChatGoogleGenerativeAI(model=MODEL,temperature=temp)
    chain_1 = LLMChain(llm=llm, prompt=SUMMARY2REQUIREMENTS_ZS, output_key="requirements")
    chain_2 = LLMChain(llm=llm, prompt=REQUIREMENTS2IMPLEMENTATION_ZS, output_key="implementation")
    chain_3 = LLMChain(llm=llm, prompt=IMPLEMETATION2STRUCTURE_ZS, output_key="structure")
    chain_4 = LLMChain(llm=llm, prompt=STRUCTURE2CODE_ZS, output_key="code")

    seq_chain = SequentialChain(
        chains = [chain_1,chain_2,chain_3,chain_4],
        input_variables=["summary"])
    
    return(seq_chain.run(summary, callbacks=[handler]))


In [4]:
from prompts import SUMMARY2REQUIRMENTS_FS, REQUIREMENTS2IMPLEMENTATION_FS, IMPLEMETATION2STRUCTURE_FS, STRUCTURE2CODE_FS

#PD_FS
def pd_fs(summary, temp=0.7):
    llm = ChatGoogleGenerativeAI(model=MODEL,temperature=temp)
    chain_1 = LLMChain(llm=llm, prompt=SUMMARY2REQUIRMENTS_FS, output_key="requirements")
    chain_2 = LLMChain(llm=llm, prompt=REQUIREMENTS2IMPLEMENTATION_FS, output_key="implementation")
    chain_3 = LLMChain(llm=llm, prompt=IMPLEMETATION2STRUCTURE_FS, output_key="structure")
    chain_4 = LLMChain(llm=llm, prompt=STRUCTURE2CODE_FS, output_key="code")

    seq_chain = SequentialChain(
        chains = [chain_1,chain_2,chain_3,chain_4],
        input_variables=["summary"])
    
    return(seq_chain.run(summary, callbacks=[handler]))

In [5]:
from prompts import SUMMARY2CODE_ZS

#Instruction
def instruction(summary, temp=0.7):
    llm = ChatGoogleGenerativeAI(model=MODEL,temperature=temp)
    chain = LLMChain(llm=llm, prompt=SUMMARY2CODE_ZS, callbacks=[handler])

    try: 
        response = chain.run(summary=summary, callbacks=[handler])
    except Exception as e:
        print(e)
    
    return(response)

In [6]:
from prompts import CoT_ZS

#CoT_ZS
def cot_zs(summary, temp=0.7):
    llm = ChatGoogleGenerativeAI(model=MODEL,temperature=temp)
    chain = LLMChain(llm=llm, prompt=CoT_ZS, callbacks=[handler])

    try: 
        response = chain.run(summary=summary, callbacks=[handler])
    except Exception as e:
        print(e)
    
    return(response)

In [7]:
from prompts import CoT_FS

#CoT_FS
def cot_fs(summary, temp=0.7):
    llm = ChatGoogleGenerativeAI(model=MODEL,temperature=temp)
    chain = LLMChain(llm=llm, prompt=CoT_FS, callbacks=[handler])

    try: 
        response = chain.run(summary=summary, callbacks=[handler])
    except Exception as e:
        print(e)
    
    return(response)

In [8]:
import re

def extract_html(code, file, folder):
    html_content = re.search(r'<!DOCTYPE html>.*</html>', code, re.DOTALL)
    extracted_text = html_content.group()
    try:
        with open(f"{folder}/{file}.html", 'w') as f:
            f.write(extracted_text)
    except Exception as e:
        print(f"{file} failed")
        print(e)

In [9]:
def prototyping(summary,folder):
   
    code = instruction(summary)
    extract_html(code,"instruction",folder)    
    
    code = cot_zs(summary)
    extract_html(code,"CoT_zs",folder)    
    
    code = cot_fs(summary)
    extract_html(code,"CoT_fs",folder)     
    
    code = pd_zs(summary)
    extract_html(code,"pd_zs",folder)    

    code = pd_fs(summary)
    extract_html(code,"pd_fs",folder)    
   


In [10]:
def pipeline():
    for index, row in df.iterrows():
        os.makedirs(f"{OUTPUT_FOLDER}/{row['UI Number']}")
        prototyping(row['Summary'],f"{OUTPUT_FOLDER}/{row['UI Number']}")


In [12]:
pipeline()

C:\Users\Marija\AppData\Local\Temp\ipykernel_4752\3960553617.py:6: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=SUMMARY2CODE_ZS, callbacks=[handler])
C:\Users\Marija\AppData\Local\Temp\ipykernel_4752\3960553617.py:9: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(summary=summary, callbacks=[handler])


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'generated_guis/15711'